# Introduction
Welcome to the "M5 Forecasting - Accuracy" competition! In this competition, contestants are challenged to forecast future sales at Walmart based on heirarchical sales in the states of California, Texas, and Wisconsin.

# Task in hand
In this competition, we need to forecast the sales for [d_1942 - d_1969]. These rows form the test set.

The rows  [d_1914 - d_1941] form the validation set.

Remaining rows form the training set.

    This notebook covers Modelling only, to check EDA, check https://www.kaggle.com/jagdmir/m5-forecasting-part-one-eda.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import gc
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,Activation,Dropout
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,LSTM
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler

# Load Data

In [ ]:
# load data
train = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
sell_prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
sample = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
train.shape,calendar.shape,sell_prices.shape

In [ ]:
train.info()

In [ ]:
calendar.info()

# Let's take a sneak peek of the data

In [ ]:
train.head()

In [ ]:
train.drop(['item_id','dept_id','cat_id','store_id','state_id'],1,inplace=True)

In [ ]:
calendar.head()

In [ ]:
sell_prices.head()

# Check Null Values

In [ ]:
train.isnull().sum().sort_values(ascending = False)

Lots of zeros above shows particular item was either not sold on that particular day or was not in stock

In [ ]:
calendar.isnull().sum().sort_values(ascending = False)

# Memory Usage Reduction

We need to melt the dataset in order to proceed further. but before we do that, we need to reduce the memory usage. if we dont reduce memory usage, we may get memory usage errors.

In [ ]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [ ]:
train = downcast(train)
sell_prices = downcast(sell_prices)
calendar = downcast(calendar)

In [ ]:
for i in range(1942,1970):
    col = "d_"+ str(i)
    train[col] = 0

In [ ]:
train_new = train.T
train_new.shape

In [ ]:
train_new = train_new[1:]
train_new

In [ ]:
train_new.shape

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
train_new = sc.fit_transform(train_new)

In [ ]:
len(train_new)

In [ ]:
# training data, from 0 to 1913
X = []
lookup = 14
for i in range(0,1899): 
    X.append(train_new[i:i+lookup])

In [ ]:
j=0
y=[]
for i in range(lookup,1913):    
    y.append(train_new[i][0:30490])
print(len(y))

In [ ]:
# Test train split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train = np.array(X_train, dtype = 'float16')
y_train = np.array(y_train, dtype = 'float16')

In [ ]:
X_train.shape,y_train.shape

In [ ]:
from keras.layers import LSTM
from keras.layers import GRU

model = Sequential()

model.add(GRU(64,input_shape=(np.array(X_train).shape[1], np.array(X_train).shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(30490))


In [ ]:
model.compile(
  loss='mse',
  metrics=[tf.keras.metrics.MeanSquaredError()],
  optimizer=tf.keras.optimizers.Adam(0.001)
)
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
epochs=20,
batch_size=10,
validation_split=0.2)

In [ ]:
acc = history.history['mean_squared_error']
val_acc = history.history['val_mean_squared_error']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Prediction on X_test

In [ ]:
from sklearn.metrics import mean_squared_error

X_test_1 = np.array(X_test[0:14])
y_test_1 = np.array(y_test[0:14])

y_test_pred = np.array(model.predict(X_test_1))

y_test_pred = sc.inverse_transform(y_test_pred)
y_test_1 = sc.inverse_transform(y_test_1)

mean_squared_error(y_test_1,y_test_pred)

# Predict for validation time frame

In [ ]:
# validation data, from d_1914 - d_1941
val = []
lookup = 14
for i in range(1913,1941):  
    val.append(train_new[i:i+lookup])

In [ ]:
np.array(val).shape

In [ ]:
np.array(y_val).shape

In [ ]:
val1 = np.array(val[:14])
val1.shape

In [ ]:
val2 = np.array(val[14:])
val2.shape

In [ ]:
y_val1_pred = model.predict(val1)
np.array(y_val1_pred).shape

In [ ]:
y_val2_pred = model.predict(val2)
np.array(y_val2_pred).shape

In [ ]:
y_val_pred = np.concatenate((y_val1_pred,y_val2_pred))
np.array(y_val_pred).shape

In [ ]:
y_val_pred = sc.inverse_transform(y_val_pred)

# Predict for test time frame

In [ ]:
test =y_val_pred.reshape(28,1,30490)
np.array(test).shape

In [ ]:
test1 = test[0:14]
np.array(test1).shape

In [ ]:
test2 = test[14:]
np.array(test2).shape

In [ ]:
test_pred1 = model.predict(test1)

In [ ]:
test_pred2 = model.predict(test2)

In [ ]:
test_pred = np.concatenate((test_pred1,test_pred2))
np.array(test_pred).shape

In [ ]:
test_pred = sc.inverse_transform(test_pred)

# Submission

In [ ]:
sub = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
sub.head()

In [ ]:
sub_ids = sub.id
sub.drop('id',1,inplace=True)

In [ ]:
sub.shape

In [ ]:
sub[0:30490] = test_pred[0:30490].T

In [ ]:
sub[30490:60980] = test_pred[0:30490].T

In [ ]:
sub.insert(loc=0, column='id', value=sub_ids)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv',index=False)